<a href="https://colab.research.google.com/github/Nishan-Charlie/AA-NoteBooks/blob/main/Yoga_Position.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import torch
import math
import os

# Clone the ultralytics repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .

# Load YOLOv8 pose model from local repository
# model = torch.hub.load('ultralytics/ultralytics', 'yolov8s-pose', pretrained=True)
from ultralytics import YOLO
model = YOLO('yolov8s-pose.pt')


# Video path
video_path = "/content/drive/MyDrive/Research/10 min Yoga for Beginners - Gentle & Simple Yoga Stretch.mp4"
output_video_path = "output_yoga_pose.mp4"

# Function to calculate angle between three points
def calculate_angle(p1, p2, p3):
    if p1 is None or p2 is None or p3 is None:
        return 0
    # Convert to numpy arrays if not already
    p1 = np.array(p1)
    p2 = np.array(p2)
    p3 = np.array(p3)

    # Vectors
    v1 = p1 - p2
    v2 = p3 - p2

    # Dot product
    dot = np.dot(v1, v2)

    # Magnitudes
    mag1 = np.linalg.norm(v1)
    mag2 = np.linalg.norm(v2)

    if mag1 == 0 or mag2 == 0:
        return 0

    # Cosine of angle
    cos = dot / (mag1 * mag2)
    cos = np.clip(cos, -1.0, 1.0)

    # Angle in degrees
    angle = math.degrees(math.acos(cos))
    return angle

# Skeleton connections for COCO keypoints
connections = [
    (5, 7), (7, 9), (6, 8), (8, 10),  # Arms
    (11, 13), (13, 15), (12, 14), (14, 16),  # Legs
    (5, 6), (5, 11), (6, 12), (11, 12),  # Torso
    (0, 1), (0, 2), (1, 3), (2, 4), (0, 5), (0, 6)  # Head
]

# Function to process frame
def process_frame(frame):
    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect poses
    results = model(rgb_frame)
    # Get detections
    if hasattr(results[0], 'keypoints'):
        keypoints_list = results[0].keypoints.xy.cpu().numpy()  # (num_persons, 17, 2)
        boxes = results[0].boxes.xyxy.cpu().numpy()
        confs = results[0].boxes.conf.cpu().numpy()
    else:
        return frame  # No detection

    for idx, (kpts, box, conf) in enumerate(zip(keypoints_list, boxes, confs)):
        if conf < 0.5:
            continue
        # Draw bounding box
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw skeleton
        for conn in connections:
            pt1 = (int(kpts[conn[0]][0]), int(kpts[conn[0]][1])) if kpts[conn[0]][0] > 0 else None
            pt2 = (int(kpts[conn[1]][0]), int(kpts[conn[1]][1])) if kpts[conn[1]][0] > 0 else None
            if pt1 and pt2:
                # Check if keypoints are within the frame boundaries before drawing
                if 0 <= pt1[0] < frame.shape[1] and 0 <= pt1[1] < frame.shape[0] and \
                   0 <= pt2[0] < frame.shape[1] and 0 <= pt2[1] < frame.shape[0]:
                    cv2.line(frame, pt1, pt2, (255, 0, 0), 2)


        # Calculate and overlay angles
        # Left elbow angle: shoulder(5)-elbow(7)-wrist(9)
        left_elbow_angle = calculate_angle(kpts[5], kpts[7], kpts[9])
        cv2.putText(frame, f"L Elbow: {left_elbow_angle:.1f}", (x1, y1 - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Right elbow angle: shoulder(6)-elbow(8)-wrist(10)
        right_elbow_angle = calculate_angle(kpts[6], kpts[8], kpts[10])
        cv2.putText(frame, f"R Elbow: {right_elbow_angle:.1f}", (x1, y1 - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Left knee angle: hip(11)-knee(13)-ankle(15)
        left_knee_angle = calculate_angle(kpts[11], kpts[13], kpts[15])
        cv2.putText(frame, f"L Knee: {left_knee_angle:.1f}", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Right knee angle: hip(12)-knee(14)-ankle(16)
        right_knee_angle = calculate_angle(kpts[12], kpts[14], kpts[16])
        cv2.putText(frame, f"R Knee: {right_knee_angle:.1f}", (x1, y1 - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Left hip angle: shoulder(5)-hip(11)-knee(13)
        left_hip_angle = calculate_angle(kpts[5], kpts[11], kpts[13])
        cv2.putText(frame, f"L Hip: {left_hip_angle:.1f}", (x1 + 150, y1 - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Right hip angle: shoulder(6)-hip(12)-knee(14)
        right_hip_angle = calculate_angle(kpts[6], kpts[12], kpts[14])
        cv2.putText(frame, f"R Hip: {right_hip_angle:.1f}", (x1 + 150, y1 - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


    return frame

# Main function for video processing
def main():
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Output video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process frame
        processed_frame = process_frame(frame)

        # Write to output
        out.write(processed_frame)

        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    out.release()
    print("Output video saved to", output_video_path)

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
0: 384x640 1 person, 409.7ms
Speed: 4.0ms preprocess, 409.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 396.0ms
Speed: 4.1ms preprocess, 396.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 402.4ms
Speed: 4.1ms preprocess, 402.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 398.8ms
Speed: 4.8ms preprocess, 398.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 395.1ms
Speed: 4.2ms preprocess, 395.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 409.3ms
Speed: 4.7ms preprocess, 409.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 394.4ms
Speed: 4.1ms preprocess, 394.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 409.8ms
Speed: 4.3ms preprocess,

In [ ]:
import shutil
import os

# Input video path
video_path = "/content/drive/MyDrive/Research/10 min Yoga for Beginners - Gentle & Simple Yoga Stretch.mp4"

# Output video originally saved here
output_video_path = "output_yoga_pose.mp4"   # current working directory

# Destination folder (same as input video)
dest_dir = os.path.dirname(video_path)
dest_path = os.path.join(dest_dir, "output_yoga_pose.mp4")

# Copy file
shutil.copy(output_video_path, dest_path)

print(f"Copied output video to: {dest_path}")


Copied output video to: /content/drive/MyDrive/Research/output_yoga_pose.mp4


In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

# Path to the output video
output_video_path = "/content/ultralytics/output_yoga_pose.mp4" # Update this if the path is different

# Create a directory to save screenshots
screenshot_dir = "screenshots"
os.makedirs(screenshot_dir, exist_ok=True)

cap = cv2.VideoCapture(output_video_path)

if not cap.isOpened():
    print("Error opening video file")
else:
    frame_count = 0
    screenshot_interval = 1000 # Save a screenshot every 100 frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Display the frame (optional, can be slow for large videos)
        # cv2_imshow(frame)

        # Save screenshot at intervals
        if frame_count % screenshot_interval == 0:
            screenshot_path = os.path.join(screenshot_dir, f"frame_{frame_count:05d}.png")
            cv2.imwrite(screenshot_path, frame)
            print(f"Saved screenshot: {screenshot_path}")

        frame_count += 1

        # Press 'q' to exit the video display
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    cap.release()
    # cv2.destroyAllWindows() # Not needed with cv2_imshow

print("Video processing complete. Screenshots saved to:", screenshot_dir)

Saved screenshot: screenshots/frame_00000.png
Saved screenshot: screenshots/frame_01000.png
Saved screenshot: screenshots/frame_02000.png
Saved screenshot: screenshots/frame_03000.png
Saved screenshot: screenshots/frame_04000.png
Saved screenshot: screenshots/frame_05000.png
Saved screenshot: screenshots/frame_06000.png
Saved screenshot: screenshots/frame_07000.png
Saved screenshot: screenshots/frame_08000.png
Saved screenshot: screenshots/frame_09000.png
Saved screenshot: screenshots/frame_10000.png
Saved screenshot: screenshots/frame_11000.png
Saved screenshot: screenshots/frame_12000.png
Saved screenshot: screenshots/frame_13000.png
Saved screenshot: screenshots/frame_14000.png
Saved screenshot: screenshots/frame_15000.png
Saved screenshot: screenshots/frame_16000.png
Saved screenshot: screenshots/frame_17000.png
Saved screenshot: screenshots/frame_18000.png
Saved screenshot: screenshots/frame_19000.png
Video processing complete. Screenshots saved to: screenshots


In [ ]:
import shutil
import os

# Input video path (used to determine the destination directory in Drive)
video_path = "/content/drive/MyDrive/Research/10 min Yoga for Beginners - Gentle & Simple Yoga Stretch.mp4"

# Source directory (where screenshots are saved)
screenshot_dir = "screenshots"

# Destination directory (same as the input video)
dest_dir = os.path.dirname(video_path)
dest_path = os.path.join(dest_dir, screenshot_dir)

# Copy the directory
shutil.copytree(screenshot_dir, dest_path)

print(f"Copied screenshots to: {dest_path}")